In [1]:
import pandas as pd 
import numpy as np  

import altair as alt
from vega_datasets import data 

import warnings
warnings.filterwarnings("ignore") 

#Link to this Data Set:  
print('https://data.cdc.gov/Public-Health-Surveillance/Rates-of-COVID-19-Cases-or-Deaths-by-Age-Group-and/d6p8-wqjm')

https://data.cdc.gov/Public-Health-Surveillance/Rates-of-COVID-19-Cases-or-Deaths-by-Age-Group-and/d6p8-wqjm


### Reading in DF created from Data Preperation

In [2]:
TSdeath_df = pd.read_csv('Time-Series Death Data.csv') 
TScase_df = pd.read_csv('Time-Series Case Data.csv')  
vaxPop_df = pd.read_csv('Time-Series Population Data.csv')
   
county_vax_df = pd.read_csv('covid-death-data.csv') 

In [3]:
pop_color_mapping = {
    'unvaccinated_population': 'red',
    'boosted_population': 'blue',
    'primary_series_only_population': 'green'
}  

In [4]:
case_color_mapping = {
    'Unvaccinated Positive Case': 'red',
    'Boosted Positive Case': 'blue',
    'Primary Series Positive Case': 'green'
}  

In [5]:
death_color_mapping = {
    'Unvaccinated Covid Mortalities': 'red',
    'Boosted Covid Mortalities': 'blue',
    'Primary Series Covid Mortalities': 'green'
}  

## The DashBoard 

In [6]:


counties = alt.topo_feature(data.us_10m.url, 'counties')
states = alt.topo_feature(data.us_10m.url, 'states')
source = county_vax_df


age_group = ['18-49', '50-64', '65+', 'all_ages']   



age_dropdown = alt.binding_select(options=age_group, name='Select Age Group')


age_selection = alt.selection_single(fields=['age_group'], bind=age_dropdown)



line = alt.Chart(TSdeath_df.reset_index()).mark_line().encode(
    x = 'date',
    y = alt.Y('Weekly Covid Mortality Totals:Q',title='Weekly Total Covid Related Mortalities'), 
    color = alt.Color('Vaccination Status:N',scale=alt.Scale(domain=list(death_color_mapping.keys()), 
                                                              range=list(death_color_mapping.values()))),  
).properties(selection=alt.selection_interval(bind='scales', zoom=False, translate=False),
    title=['Deaths Totals per Week by Vaccinated Population']  
).add_selection( 
    age_selection 
).transform_filter( 
    age_selection   
).interactive()
 
                           
line2 = alt.Chart(TScase_df.reset_index()).mark_line().encode(
    x = 'date',
    y = alt.Y('Weekly Covid Positive Case Totals:Q', title='Weekly Total Positive Covid Cases'), 
    color = alt.Color('Vaccination Status:N', scale=alt.Scale(domain=list(case_color_mapping.keys()), 
                                                              range=list(case_color_mapping.values()))), 
).properties(selection=alt.selection_interval(bind='scales', zoom=False, translate=False), 
             title = ['Case Totals per Week by Vaccinated Population']
).add_selection( 
    age_selection  
).transform_filter( 
    age_selection   
).interactive()  

area_chart = alt.Chart(vaxPop_df).mark_area(opacity=0.1).encode(
    x= "date",
    y= alt.Y('Percentage of Total Population by Vaccination Status:Q',title='% of Population (Ages 18-49)', 
             scale=alt.Scale(type='linear'),stack='normalize'), 
    color = alt.Color('Vaccination Status:N', scale=alt.Scale(domain=list(pop_color_mapping.keys()), 
                                                              range=list(pop_color_mapping.values())))
).properties(selection=alt.selection_interval(bind='scales', zoom=False, translate=False), 
             title = ['Weekly Population for Vaccination Status Groups'], 
)



highlight = alt.selection_single(on='mouseover', fields=['id'], empty='none')

plot = alt.Chart(counties).mark_geoshape().encode(
    color=alt.condition(highlight, alt.value('red'), 'covid_deaths_per_500k:Q'),
    tooltip=['covid_deaths_per_500k:Q', 'county:N']
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(source, 'county_fips', ['covid_deaths_per_500k', 'county'])
).add_selection(highlight).project(
    type='albersUsa'
).properties(
    width=900,
    height=600,
)

outline = alt.Chart(states).mark_geoshape(stroke='blue', fillOpacity=0).project(
    type='albersUsa'
).properties(
    width=700,
    height=400
)

m = alt.layer(plot,outline).properties(title=["Covid Related Deaths per 500k by County"])   


In [12]:
Layer_case = alt.layer(line2, area_chart).resolve_scale(y='independent').properties(width=800, height=400) 
Layer_death = alt.layer(line, area_chart).resolve_scale(y='independent').properties(width=800,height=400)

In [ ]:
final_plots = m & Layer_case & Layer_death

In [ ]:
final_plots

!pip install altair_saver

import altair_saver

#Save the chart as an HTML file
altair_saver.save(final_plots, 'Covid-19 Vaccine Efficacy Dashboard.html')

from IPython.display import HTML

#Define the file path of the HTML file
file_path = './Covid-19 Vaccine Efficacy Dashboard.html'

#Create an HTML string with a hyperlink to the HTML file
html_string = f'<a href="{file_path}" target="_blank">Open HTML file</a>'

#Display the HTML string in the notebook output
HTML(html_string)
